#Getting Started
This notebook provides a basic introduction to building a LangGraph, a framework for creating complex conversational AI agents. It covers a basic conversational loop where the assistant generates a response, a condition checks if a tool is needed, and if so, a tool is invoked, and its output is fed back to the assistant. The visualization helps in understanding this flow and the decision points within the graph.

### Install Required Packages

In [ ]:
%pip install --upgrade --user --quiet "google-cloud-aiplatform[evaluation, langchain, reasoningengine]" \
    "langchain_google_vertexai" \
    "langgraph" \
    "cloudpickle==3.0.0" \
    "pydantic>=2.10" \
    "requests"

### Restart current runtime

In [ ]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

### Authenticate your notebook environment (Colab only)

In [ ]:
from google.colab import auth
auth.authenticate_user()

### Set Google Cloud project information and initialize Vertex AI SDK

In [ ]:
# Use the environment variable if the user doesn't provide Project ID.
import os
import vertexai

PROJECT_ID =""  # @param {type: "string", placeholder: "[your-project-id]", isTemplate: true}
LOCATION ="us-central1"


vertexai.init(project=PROJECT_ID,location=LOCATION)

### Import libraries

In [ ]:
from langgraph.graph import START, StateGraph
from langgraph.prebuilt import tools_condition
from langgraph.prebuilt import ToolNode
from IPython.display import Image, display
from langchain_google_vertexai import ChatVertexAI

### Define model

In [ ]:
llm=ChatVertexAI(model_name="gemini-1.5-flash-001")

### Define Python functions (tools)

In [ ]:
def multiply(a: int, b: int) -> int:
    """Multiply a and b.

    Args:
        a: first int
        b: second int
    """
    return a * b

# This will be a tool
def add(a: int, b: int) -> int:
    """Adds a and b.

    Args:
        a: first int
        b: second int
    """
    return a + b

def divide(a: int, b: int) -> float:
    """Divide a and b.

    Args:
        a: first int
        b: second int
    """
    return a / b

tools = [add, multiply, divide]

In [ ]:
# For this ipynb we set parallel tool calling to false as math generally is done sequentially, and this time we have 3 tools that can do math
# the OpenAI model specifically defaults to parallel tool calling for efficiency, see https://python.langchain.com/docs/how_to/tool_calling_parallel/
# play around with it and see how the model behaves with math equations!
llm_with_tools = llm.bind_tools(tools, parallel_tool_calls=False)

In [ ]:
from langgraph.graph import MessagesState
from langchain_core.messages import HumanMessage, SystemMessage
# The message state acts as the memory or the history of interactions within your LangGraph application.
# It's the primary way different nodes (representing steps or agents) in the graph communicate and share information.


# System message
sys_msg = SystemMessage(content="You are a helpful assistant tasked with performing arithmetic on a set of inputs.")

# Node
def assistant(state: MessagesState):
   return {"messages": [llm_with_tools.invoke([sys_msg] + state["messages"])]}

In [ ]:
# Graph
builder = StateGraph(MessagesState)

# Define nodes: these do the work
builder.add_node("assistant", assistant)
builder.add_node("tools", ToolNode(tools))

# Define edges: these determine how the control flow moves
builder.add_edge(START, "assistant")
builder.add_conditional_edges(
    "assistant",
    tools_condition,
)

# The tools_condition being used is a pre-built function provided by LangGraph.
# This function is designed to specifically handle the logic of determining whether,
# the output of a language model ("assistant" node) indicates a need to use a tool.

builder.add_edge("tools", "assistant")
react_graph = builder.compile()

# Show
display(Image(react_graph.get_graph(xray=True).draw_mermaid_png()))

In [ ]:
messages = [HumanMessage(content="Add 3 and 4. Multiply the output by 2. Divide the output by 5")]
messages = react_graph.invoke({"messages": messages})

In [ ]:
for m in messages['messages']:
    m.pretty_print()